# Analyzing Customer Behavior for E-commerce Insights.

## Business Understanding 

E-commerce businesses generate massive amounts of event data from customer sessions, product views, and purchases. Leveraging this data is critical for improving sales, strengthening engagement, and delivering personalized shopping experiences.
Npontu Technologies collects large volumes of e-commerce event data,however the company currently lacks a structured way to translate this raw data into meaningful insights.The business wants to raise revenue, improve engagement, and tailor offers.


### Problem Statement

The company lacks clear signals to find at-risk customers, high-value buyers, and product opportunities. This gap limits the company’s ability to raise revenue, improve customer engagement, and deliver personalized experiences that keep customers coming back.


### Business Objectives 
- Increase repeat purchase rate by identifying and re-engaging at-risk customers.

- Grow average order value by targeting high-potential buyers with offers.

- Improve conversion rate by optimizing product funnels and merchandising.

- Personalize marketing to reduce churn and raise lifetime value (LTV).

- Demonstrate a scalable pipeline (streaming or batch) to compute features in near real-time.


### Goal
By leveraging on the CRISP-DM framework I will transform raw event logs into customer profiles, predictive models, and interactive dashboards. Npontu Technologies can gain actionable insights that drive revenue growth, enhance customer loyalty, and support data-driven decision-making.



### Stakeholders

- Product Manager — decides promotions, product placements, UX changes.

- Growth/Marketing Team — runs campaigns and needs segments and uplift targets.

- Customer Success Team — acts on churn predictions and win-back flows.

- Data Engineering Team — builds ingestion and feature pipelines (Kafka, Spark).

- Data Science Team — builds models and explains them.

- Business Leadership — reviews ROI, revenue impact, and prioritizes initiatives.



### Key Features

The dataset contains the following key files:

1. customers.csv

- Contains customer-level information. Fields include customer_id, age, gender, location and signup_date

2. sessions.csv

- Captures details about customer browsing sessions. This includes session_id, customer_id, session_date, product_viewed, browsing_time_sec,purchase_made and purchase_amount.This helps track how customers interact with the platform over time.




### Key Features of the E-Commerce Insights

- Churn risk list with confidence scores and top drivers per customer.

- Customer segments (new, loyal, at-risk, high LTV) and recommended actions.

- Product funnel metrics: view → add to cart → purchase conversion by product.

- Time trends and seasonality: weekly/monthly demand peaks and campaign lift.

- Top revenue cohorts and product pairs for cross-sell suggestions.

- Dashboard for monitoring model performance and business KPIs.




### Key features to engineer (data features to create)

- RFM: recency (days since last purchase), frequency (# purchases), monetary (total spend).

- Session features: avg session duration, pages viewed, product views per session.

- Recency buckets and days_since_last_purchase (for churn label).

- Behavioral rates: add-to-cart rate, purchase_rate = purchases / sessions.

- Product interaction features: top categories viewed, favorite category, cross-view counts.

- Temporal features: hour_of_day, day_of_week, month, days_since_signup, season flags.

- Derived financials: avg_order_value, CLTV proxy (sum of orders over period), discount usage.




### Hypothesis

Customers with long recency (no recent purchase) and low session frequency are more likely to churn.



### 7 Analytical Questions

1. Which customers are likely to churn in the next 90 days?

- Use recency, frequency, monetary, session patterns, and recent activity as features.

2. Which actions move at-risk customers back to buying?

- Test offers, email cadence, and personalized product recommendations (A/B test).

3. Which product categories have the largest view → purchase leakage?

- Identify pages with high views but low purchases to target UX or pricing fixes.

4. Who are the top 5% highest-value customers and what early signals identify them?

- Build a short-term predictor to flag future high-value buyers within first 30 days.

5. What is the average order value (AOV) by customer cohort and how can it be increased?

- Segment by acquisition channel, signup month, and product interest.

6. Which product combinations or sequences suggest strong cross-sell opportunities?

- Use association rules or co-view / co-purchase analysis.

7. How does conversion and purchase behavior change over time (seasonality & campaign effects)?

- Attribute lifts to marketing and note when retraining models is necessary.







### Data Understanding & Preparation
Importing all the relevant libraries

In [1]:
# Core Python Utilities & Configurations

import os
import warnings
import logging
 
warnings.filterwarnings("ignore")
 
# Data Handling and Manipulation
import pandas as pd
import numpy as np
import scipy

# Big Data Processing
from kafka import KafkaConsumer, KafkaProducer  # or just import kafka if using differently
from pyspark.sql import SparkSession

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


# Feature Engineering & Preprocessing

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
 

# Anomaly Detection & Clustering

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN
 

# Model Evaluation & Hyperparameter Tuning

from sklearn.metrics import accuracy_score, classification_report, ndcg_score
from sklearn.model_selection import GridSearchCV
 

# Performance & Progress Tracking

from tqdm import tqdm  
 

# Statistical and Scientific Computing
import statsmodels.api as sm

# Machine Learning & Recommendation Systems
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # example ML model
from sklearn.metrics import accuracy_score  # example metric

# Geographic Data & Mapping
import geopandas as gpd

# Dashboarding for Final Reporting
import dash
from dash import dcc, html

import dash_bootstrap_components as dbc


# Initialization & Status Confirmation

print("✅ Project initialized successfully! All necessary libraries are loaded.")













✅ Project initialized successfully! All necessary libraries are loaded.


### Load all datasets from their sources

In [2]:

# Path of customer csv file
file_path = '../Analyzing-Customer-Behavior-for-E-commerce-Insights/Data/customers.csv'

# Check if the file exists at the specified path
if os.path.exists(file_path):
    print("File exists at the specified path.")
    try:
        # Read the Excel file into a pandas DataFrame
        df_customers = pd.read_csv(file_path)
       
    except FileNotFoundError as e:
        print(f"FileNotFoundError: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("File does not exist at the specified path.")
 
# Display the DataFrame
df_customers.head()


File exists at the specified path.


,customer_id,age,gender,location,signup_date
0,CUST00001,56,Female,Ghana,2020-11-29
1,CUST00002,69,Male,India,2022-11-09
2,CUST00003,46,Female,NaN,2020-07-23
3,CUST00004,32,Male,India,2022-08-13
4,CUST00005,60,Female,Brazil,2020-12-20


In [3]:
# Path of session csv file
file_path = '../Analyzing-Customer-Behavior-for-E-commerce-Insights/Data/sessions.csv'

# Check if the file exists at the specified path
if os.path.exists(file_path):
    print("File exists at the specified path.")
    try:
        # Read the Excel file into a pandas DataFrame
        df_sessions = pd.read_csv(file_path)
       
    except FileNotFoundError as e:
        print(f"FileNotFoundError: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("File does not exist at the specified path.")
 
# Display the DataFrame
df_sessions.head()


File exists at the specified path.


,session_id,customer_id,session_date,product_viewed,browsing_time_sec,purchase_made,purchase_amount
0,SESS000001,CUST04899,2021-06-16,Laptop,743,0,0.0
1,SESS000002,CUST00246,2022-09-23,Laptop,247,0,0.0
2,SESS000003,CUST04945,2023-03-08,Laptop,201,0,0.0
3,SESS000004,CUST03372,2021-08-24,Phone,93,0,0.0
4,SESS000005,CUST01351,2023-07-11,Book,330,0,0.0


#### Concatenating the two data sets to form one. 
We will do a left join using the Customer Id. df_final is the combined data set of customers and sessions data

In [4]:
df_final = df_sessions.merge(df_customers, how='left', on='customer_id', suffixes=('_df_sessions','_df_customers'))



## Exploratory Data Analysis(EDA)

In [5]:
df_final.head()

,session_id,customer_id,session_date,product_viewed,browsing_time_sec,purchase_made,purchase_amount,age,gender,location,signup_date
0,SESS000001,CUST04899,2021-06-16,Laptop,743,0,0.0,31.0,Female,US,2020-04-25
1,SESS000002,CUST00246,2022-09-23,Laptop,247,0,0.0,34.0,Female,Brazil,2021-01-26
2,SESS000003,CUST04945,2023-03-08,Laptop,201,0,0.0,64.0,Male,India,2021-03-01
3,SESS000004,CUST03372,2021-08-24,Phone,93,0,0.0,52.0,Female,India,2022-01-28
4,SESS000005,CUST01351,2023-07-11,Book,330,0,0.0,57.0,Female,US,2021-04-06


In [6]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50100 entries, 0 to 50099
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   session_id         50100 non-null  object 
 1   customer_id        49900 non-null  object 
 2   session_date       50100 non-null  object 
 3   product_viewed     49099 non-null  object 
 4   browsing_time_sec  50100 non-null  int64  
 5   purchase_made      50100 non-null  int64  
 6   purchase_amount    50100 non-null  float64
 7   age                49900 non-null  float64
 8   gender             48983 non-null  object 
 9   location           47542 non-null  object 
 10  signup_date        49900 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 4.2+ MB


In [7]:
#Summary Statistics
df_final.describe()

,browsing_time_sec,purchase_made,purchase_amount,age
count,50100.000000,50100.000000,50100.000000,49900.000000
mean,300.831776,0.299521,300.315221,43.484549
std,298.183835,0.458053,557.318938,14.930561
min,0.000000,0.000000,0.000000,18.000000
25%,87.000000,0.000000,0.000000,31.000000
50%,210.000000,0.000000,0.000000,43.000000
75%,418.000000,1.000000,338.687500,56.000000
max,3316.000000,1.000000,1999.740000,69.000000


In [8]:
# Get the list of all column names in the DataFrame
columns = df_final.columns
 
# Print details of unique values for each column in the DataFrame
for column in columns:
    print(f'{column}')  
    print(f'There are {df_final[column].unique().size} unique values')  
    print(f'{df_final[column].unique()}')  
    print('_' * 80)

session_id
There are 50000 unique values
['SESS000001' 'SESS000002' 'SESS000003' ... 'SESS049998' 'SESS049999'
 'SESS050000']
________________________________________________________________________________
customer_id
There are 5001 unique values
['CUST04899' 'CUST00246' 'CUST04945' ... 'CUST01540' 'CUST01160'
 'CUST03565']
________________________________________________________________________________
session_date
There are 1000 unique values
['2021-06-16' '2022-09-23' '2023-03-08' '2021-08-24' '2023-07-11'
 '2022-02-02' '2021-11-16' '2023-03-26' '2022-06-16' '2021-01-07'
 '2022-12-23' '2022-04-03' '2021-08-26' '2023-05-12' '2022-03-27'
 '2023-06-15' '2021-04-24' '2022-05-07' '2021-11-05' '2022-03-25'
 '2021-06-07' '2023-04-14' '2022-02-03' '2022-10-01' '2021-07-22'
 '2023-01-06' '2022-08-05' '2022-03-30' '2023-05-10' '2021-04-30'
 '2023-08-11' '2022-10-27' '2022-06-19' '2022-10-14' '2023-08-02'
 '2022-09-16' '2022-10-12' '2023-07-02' '2021-06-29' '2022-10-04'
 '2022-12-01' '2022-05

In [9]:
df_final.duplicated().sum()

np.int64(100)

In [10]:
# Check for duplicated rows
duplicated_rows = df_final.duplicated()

# Display the number of duplicated rows
print(f"Number of duplicated rows: {duplicated_rows.sum()}")

# Display duplicated rows
df_customers[duplicated_rows]

Number of duplicated rows: 100


,customer_id,age,gender,location,signup_date


In [11]:
# Check for duplicated columns

# Transpose the DataFrame and check for duplicated columns
duplicated_columns = df_final.T.duplicated()

# Get the list of duplicated column names
duplicate_col_names = df_final.columns[duplicated_columns]

# Display the number and names of duplicated columns
print(f"Number of duplicated columns: {duplicated_columns.sum()}")
print(f"Duplicated columns: {list(duplicate_col_names)}")

# Optional: Drop duplicated columns if needed
# df_final = df_final.loc[:, ~duplicated_columns]


Number of duplicated columns: 0
Duplicated columns: []


In [12]:
# Check the datatype and the number of columns for customers data set 

df_final.info(verbose=False, memory_usage='deep')
 
missing_counts = df_final.isna().sum()
non_null_counts = df_final.notna().sum()
dtype_info = df_final.dtypes
 
df_finalinfo = pd.DataFrame({
    "Non-Null Count": non_null_counts,
    "Missing Count": missing_counts,
    "Missing Percentage": round((missing_counts / len(df_final)) * 100, 2),
    "Dtype": dtype_info
})
 
# Display results
df_finalinfo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50100 entries, 0 to 50099
Columns: 11 entries, session_id to signup_date
dtypes: float64(2), int64(2), object(7)
memory usage: 20.4 MB


,Non-Null Count,Missing Count,Missing Percentage,Dtype
session_id,50100,0,0.00,object
customer_id,49900,200,0.40,object
session_date,50100,0,0.00,object
product_viewed,49099,1001,2.00,object
browsing_time_sec,50100,0,0.00,int64
purchase_made,50100,0,0.00,int64
purchase_amount,50100,0,0.00,float64
age,49900,200,0.40,float64
gender,48983,1117,2.23,object
location,47542,2558,5.11,object
